<a href="https://colab.research.google.com/github/janvierk/MRP/blob/main/MRP_mBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

In [4]:
os.chdir('/content/drive/MyDrive/MRP')

In [6]:
#installing requitred libries and supressing  the print-out
#!pip install requests==2.31.0 --upgrade -q > NUL
!pip install accelerate -U -q > NUL
!pip install transformers -q > NUL
!pip install torch -q > NUL
#!pip install -U transformers -q >NULip install datasets -q > NUL
!pip install tensorflow -q > NUL
!pip install datasets -q > NUL
!pip install evaluate -q > NUL

In [7]:
#Required packages and libries
#Required packages and libries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers  import Trainer
from transformers import TrainingArguments
from transformers import XLMRobertaTokenizer
from transformers import XLMRobertaForSequenceClassification
from transformers import MT5ForConditionalGeneration
from transformers import T5Tokenizer
import seaborn as sn
import matplotlib
from transformers import AutoTokenizer, AutoModelForTokenClassification
import transformers as tf
import torch
from datasets import load_dataset
import datasets
import  evaluate

In [8]:
#importing the clean dataset to train the model
swahili_dat = pd.read_csv("swahili_clean.csv")

In [9]:
english_dat = pd.read_csv("english_news_clean")

In [10]:
english_dat.shape[0]

29544

In [11]:
swahili_dat.shape[0]

29544

In [12]:
english_dat.shape[0]

29544

In [13]:
#shuffling the datasets
swahili_dat= swahili_dat.sample(frac =1, random_state =42)
english_dat = english_dat.sample(frac =1, random_state = 42)

In [14]:
english_dat.value_counts("label")

,count
label,
0,16000
1,11720
2,854
3,557
4,413


In [ ]:

#creating a small swahili dataset

In [15]:
#dividing swahili dataset for training, development and testing
from sklearn.model_selection import train_test_split
swahili_train, swahili_development = train_test_split(swahili_dat, test_size=0.4,  random_state = 42 )
swahili_validation, swahili_testing = train_test_split(swahili_development, test_size = 0.5, random_state = 42)

In [16]:
#resetting the index
swahili_testing.reset_index(drop = True, inplace = True)
swahili_validation.reset_index(drop = True, inplace = True)
swahili_train.reset_index(drop = True, inplace = True)

In [17]:
#saving the dataset
swahili_testing.to_csv("swahili_testing.csv", index=False)
swahili_train.to_csv("swahili_train.csv", index =False)
swahili_validation.to_csv("swahili_validation.csv", index = False)

In [18]:
#dividing cnn dataset for training, validation and testing
english_train, english_development = train_test_split(english_dat, test_size=0.4,  random_state = 42 )
english_validation, english_testing = train_test_split(english_development, test_size = 0.5, random_state = 42)

In [19]:
#resetting the index
english_testing.reset_index(drop = True, inplace = True)
english_validation.reset_index(drop = True, inplace = True)
english_train.reset_index(drop = True, inplace = True)

In [20]:
#saving the english dataset
english_testing.to_csv("english_testing.csv", index=False)
english_train.to_csv("english_train.csv", index =False)
english_validation.to_csv("english_validation.csv", index = False)

In [21]:
#Uisng Bert tokenizer for tokernization of the data
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

def tokenize_data(text):
  return tokenizer(text,
                   padding = "max_length",
                   truncation = True,
  )


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [22]:
#tokenizing swahili data
tokenized_swahili_train =  swahili_train["text"].apply(tokenize_data)
tokenized_swahili_validation = swahili_validation["text"].apply(tokenize_data)
tokenized_swahili_testing = swahili_testing["text"].apply(tokenize_data)

In [23]:
#tokenizing english data
tokenized_english_train =  english_train["text"].apply(tokenize_data)
tokenized_english_validation = english_validation["text"].apply(tokenize_data)
tokenized_english_testing = english_testing["text"].apply(tokenize_data)

In [24]:
#checking if the text has been tokenized accordingly
for i in range(3):
    print(f"Tokenized entry {i+1}:")
    print(f"Input IDs: {tokenized_swahili_train.iloc[i]['input_ids']}")
    print(f"Token Type IDs: {tokenized_swahili_train.iloc[i]['token_type_ids']}")
    print(f"Attention Mask: {tokenized_swahili_train.iloc[i]['attention_mask']}")
    print("\n")

Tokenized entry 1:
Input IDs: [101, 28941, 18845, 10116, 18939, 103695, 10537, 10879, 69553, 26127, 10113, 14876, 10465, 19502, 10133, 15176, 12589, 10879, 69553, 13024, 10381, 10113, 10879, 35481, 37861, 10113, 99946, 10401, 107591, 20338, 10132, 10883, 16423, 10908, 28532, 52866, 95342, 10549, 106306, 11044, 10109, 44795, 15914, 15529, 10133, 13171, 10559, 102161, 11471, 69553, 11952, 76372, 13723, 58671, 11471, 21511, 14590, 11037, 39796, 10549, 11856, 14383, 64470, 13605, 10133, 33376, 10219, 10132, 106712, 10550, 10133, 10549, 10911, 12806, 10549, 23605, 10116, 10549, 10148, 10246, 10549, 123, 118, 123, 28532, 158, 14394, 10320, 11471, 25633, 13369, 117, 35368, 119, 13026, 28167, 11471, 12865, 49581, 11460, 10228, 10133, 117, 11118, 10820, 10392, 13565, 10369, 117, 11471, 41947, 78314, 19830, 82125, 10113, 14876, 18089, 15694, 23661, 86137, 13133, 11282, 13605, 10133, 14266, 15694, 37583, 12177, 10113, 189, 81111, 10775, 11471, 10133, 11471, 75459, 10376, 11037, 11471, 32874, 4986

In [25]:
#extracting input ids and attention mask for swahili training
input_ids_swahili_train= [item["input_ids"] for item in tokenized_swahili_train]
attention_mask_swahili_train = [item["attention_mask"] for item in tokenized_swahili_train]
swahili_encodings_train = {"input_ids": input_ids_swahili_train,
             "attention_mask": attention_mask_swahili_train}

In [26]:
#extracting input ids and attention mask for swahili validation
input_ids_swahili_validation= [item["input_ids"] for item in tokenized_swahili_validation]
attention_mask_swahili_validation = [item["attention_mask"] for item in tokenized_swahili_validation]
swahili_encodings_validation = {"input_ids": input_ids_swahili_validation,
             "attention_mask": attention_mask_swahili_validation}

In [27]:
#extracting input ids and attention mask for swahili testing
input_ids_swahili_testing= [item["input_ids"] for item in tokenized_swahili_testing]
attention_mask_swahili_testing = [item["attention_mask"] for item in tokenized_swahili_testing]
swahili_encodings_testing = {"input_ids": input_ids_swahili_testing,
             "attention_mask": attention_mask_swahili_testing}

In [28]:
#extracting input ids and attention mask for english training
input_ids_english_train= [item["input_ids"] for item in tokenized_english_train]
attention_mask_english_train = [item["attention_mask"] for item in tokenized_english_train]
english_encodings_train = {"input_ids": input_ids_english_train,
             "attention_mask": attention_mask_english_train}

In [29]:
#extracting input ids and attention mask for english validation
input_ids_english_validation= [item["input_ids"] for item in tokenized_english_validation]
attention_mask_english_validation = [item["attention_mask"] for item in tokenized_english_validation]
english_encodings_validation = {"input_ids": input_ids_english_validation,
             "attention_mask": attention_mask_english_validation}

In [30]:
#extracting input ids and attention mask for english testing
input_ids_english_testing= [item["input_ids"] for item in tokenized_english_testing]
attention_mask_english_testing = [item["attention_mask"] for item in tokenized_english_testing]
english_encodings_testing = {"input_ids": input_ids_english_testing,
             "attention_mask": attention_mask_english_testing}

In [31]:
#getting labels for swahili datasets
swahili_train_labels = swahili_train["label"]
swahili_validation_labels = swahili_validation["label"]
swahili_testing_labels = swahili_testing["label"]

In [32]:
#getting labels for english datasets
english_train_labels = english_train["label"]
english_validation_labels = english_validation["label"]
english_testing_labels = english_testing["label"]

In [33]:
#creating tensor data to feed into the model
import torch
from torch.utils.data import Dataset, DataLoader

class TextDataset (Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item["labels"] = torch.tensor(self.labels[idx])
    return item
  def __len__(self):
    return len(self.labels)

In [34]:
#swahili ready data
#english ready data
swahili_ready_train = TextDataset(swahili_encodings_train, swahili_train_labels)
swahili_ready_validation = TextDataset(swahili_encodings_validation, swahili_validation_labels)
swahili_ready_testing = TextDataset(swahili_encodings_testing, swahili_testing_labels)

In [35]:
#english ready data
english_ready_train = TextDataset(english_encodings_train, english_train_labels)
english_ready_validation = TextDataset(english_encodings_validation, english_validation_labels)
english_ready_testing = TextDataset(english_encodings_testing, english_testing_labels)

In [36]:
def inspect_dataset(dataset, num_samples):
  for i in range(min(num_samples, len(dataset))):
    item = dataset[i]
    print(f"samples{i}:")
    for key, value in item.items():
      print(f" {key} : {value}")
    print("\n")


In [37]:
#Building  a simple mBert model for small swahili dataset
!pip install -U accelerate -q >NUL
!pip install -U transformers -q >NUL
swahili_mbert = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased",
                                                                  num_labels = swahili_train["label"].nunique())
english_mbert =  BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased",
                                                                  num_labels = english_train["label"].nunique())

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
#training arguments
training_args_sm = TrainingArguments(
    output_dir="./sm_results1",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=200,
    weight_decay=0.01,
    logging_dir="./sm_logging1",
    logging_steps=20,
    load_best_model_at_end=True,
    eval_strategy="epoch",
    save_strategy="epoch"
)
training_args_em = TrainingArguments(
    output_dir="./em_results1",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=200,
    weight_decay=0.01,
    logging_dir="./em_logging1",
    logging_steps=20,
    load_best_model_at_end=True,
    eval_strategy="epoch",
    save_strategy="epoch"
)

In [39]:
#computing the evaluation metrics
import evaluate
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")


In [40]:
from transformers import EarlyStoppingCallback
from sklearn.metrics import confusion_matrix

In [ ]:
"""#computing the evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    accuracy = accuracy_metric.compute(predictions=preds, references=labels)
    f1 = f1_metric.compute(predictions=preds, references=labels, average='weighted')
    precision = precision_metric.compute(predictions=preds, references=labels, average='weighted')
    recall = recall_metric.compute(predictions=preds, references=labels, average='weighted')

    return {
        'accuracy': accuracy['accuracy'],
        'f1': f1['f1'],
        'precision': precision['precision'],
        'recall': recall['recall']
    }"""

In [41]:
#computing the evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    accuracy = accuracy_metric.compute(predictions=preds, references=labels)
    f1 = f1_metric.compute(predictions=preds, references=labels, average='weighted')
    precision = precision_metric.compute(predictions=preds, references=labels, average='weighted')
    recall = recall_metric.compute(predictions=preds, references=labels, average='weighted')
    conf_matrix = confusion_matrix(labels, preds)
    conf_matrix_list = conf_matrix.tolist()

    return {
        'accuracy': accuracy['accuracy'],
        'f1': f1['f1'],
        'precision': precision['precision'],
        'recall': recall['recall'],
        'conf_matrix': conf_matrix_list
    }

In [42]:
import torch
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import torch.nn as nn
# Calculate class weights using sklearn's compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(swahili_train_labels), y=swahili_train_labels)

# Convert class weights to a tensor for PyTorch
class_weights = torch.tensor(class_weights, dtype=torch.float)

In [43]:
#Define your custom Trainer class with class weights
class CustomTrainer(Trainer):
    def __init__(self, class_weights, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights.to(self.args.device)  # Move weights to the appropriate device

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Define the loss function with class weights
        loss_fct = nn.CrossEntropyLoss(weight=self.class_weights)
        loss = loss_fct(logits, labels)

        return (loss, outputs) if return_outputs else loss

In [44]:
mbert_trainer_swahili = CustomTrainer(
    model=swahili_mbert,
    args=training_args_sm,
    train_dataset=swahili_ready_train,
    eval_dataset=swahili_ready_validation,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)],
    class_weights=class_weights)  # Pass class weights to the custom trainer

In [ ]:
"""mbert_trainer_swahili= Trainer(
    model=swahili_mbert,
    args=training_args_sm,
    train_dataset=swahili_ready_train,
    eval_dataset=swahili_ready_validation,
    compute_metrics=compute_metrics,  # Use the modified compute_metrics function
)"""

In [ ]:
"""mbert_trainer_english = Trainer(
    model=english_mbert,
    args=training_args_em,
    train_dataset=english_ready_train,
    eval_dataset=english_ready_validation,
    compute_metrics=compute_metrics,  # Use the modified compute_metrics function
)"""

In [45]:
mbert_trainer_english = CustomTrainer(
    model=english_mbert,
    args=training_args_em,
    train_dataset=english_ready_train,
    eval_dataset=english_ready_validation,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)],
    class_weights=class_weights)  # Pass class weights to the custom trainer

In [46]:
mbert_trainer_swahili.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Conf Matrix
1,0.551900,0.602845,0.866813,0.868223,0.873432,0.866813,"[[2757, 79, 63, 153, 163], [105, 215, 0, 0, 18], [15, 1, 490, 5, 41], [78, 4, 4, 104, 10], [20, 6, 20, 2, 1556]]"
2,0.472800,0.495836,0.912168,0.911185,0.911537,0.912168,"[[2976, 31, 43, 75, 90], [89, 238, 0, 0, 11], [18, 0, 495, 6, 33], [75, 0, 8, 117, 0], [21, 8, 11, 0, 1564]]"
3,0.392100,0.489622,0.912506,0.912472,0.913167,0.912506,"[[2929, 81, 49, 79, 77], [69, 260, 0, 0, 9], [11, 1, 516, 1, 23], [76, 1, 5, 118, 0], [17, 6, 12, 0, 1569]]"


Trainer is attempting to log a value of "[[2757, 79, 63, 153, 163], [105, 215, 0, 0, 18], [15, 1, 490, 5, 41], [78, 4, 4, 104, 10], [20, 6, 20, 2, 1556]]" of type <class 'list'> for key "eval/conf_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[2976, 31, 43, 75, 90], [89, 238, 0, 0, 11], [18, 0, 495, 6, 33], [75, 0, 8, 117, 0], [21, 8, 11, 0, 1564]]" of type <class 'list'> for key "eval/conf_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[2929, 81, 49, 79, 77], [69, 260, 0, 0, 9], [11, 1, 516, 1, 23], [76, 1, 5, 118, 0], [17, 6, 12, 0, 1569]]" of type <class 'list'> for key "eval/conf_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=3324, training_loss=0.541478790961448, metrics={'train_runtime': 1277.46, 'train_samples_per_second': 41.628, 'train_steps_per_second': 2.602, 'total_flos': 1.3992096579950592e+16, 'train_loss': 0.541478790961448, 'epoch': 3.0})

In [49]:
#evaluation of swahili dataset
#testing mbert swahili model on testing dataset
mbert_test_swahili = mbert_trainer_swahili.evaluate(eval_dataset=swahili_ready_testing)

Trainer is attempting to log a value of "[[2911, 72, 29, 99, 59], [93, 277, 1, 0, 8], [16, 1, 550, 5, 20], [73, 0, 4, 118, 0], [10, 4, 25, 0, 1534]]" of type <class 'list'> for key "eval/conf_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


In [64]:
print(mbert_test_swahili)

{'eval_loss': 0.4714862108230591, 'eval_accuracy': 0.9121678795058386, 'eval_f1': 0.9125165489608603, 'eval_precision': 0.9134012996831977, 'eval_recall': 0.9121678795058386, 'eval_conf_matrix': [[2911, 72, 29, 99, 59], [93, 277, 1, 0, 8], [16, 1, 550, 5, 20], [73, 0, 4, 118, 0], [10, 4, 25, 0, 1534]], 'eval_runtime': 37.9642, 'eval_samples_per_second': 155.647, 'eval_steps_per_second': 9.746, 'epoch': 3.0}


In [65]:
# Assuming mbert_test_swahili contains the evaluation results
eval_loss = mbert_test_swahili.get('eval_loss', None)
eval_accuracy = mbert_test_swahili.get('eval_accuracy', None)
eval_f1 = mbert_test_swahili.get('eval_f1', None)
eval_precision = mbert_test_swahili.get('eval_precision', None)
eval_recall = mbert_test_swahili.get('eval_recall', None)
eval_conf_matrix = mbert_test_swahili.get('eval_conf_matrix', None)


In [66]:
print(f"Evaluation Loss: {eval_loss}")
print(f"Evaluation Accuracy: {eval_accuracy}")
print(f"Evaluation F1 Score: {eval_f1}")
print(f"Evaluation Precision: {eval_precision}")
print(f"Evaluation Recall: {eval_recall}")
print(f"Confusion Matrix: {eval_conf_matrix}")

Evaluation Loss: 0.4714862108230591
Evaluation Accuracy: 0.9121678795058386
Evaluation F1 Score: 0.9125165489608603
Evaluation Precision: 0.9134012996831977
Evaluation Recall: 0.9121678795058386
Confusion Matrix: [[2911, 72, 29, 99, 59], [93, 277, 1, 0, 8], [16, 1, 550, 5, 20], [73, 0, 4, 118, 0], [10, 4, 25, 0, 1534]]


In [50]:
#accessing the log information
training_logs_swahili = mbert_trainer_swahili.state.log_history

In [80]:
#creating a

In [76]:
mbert_trainer_english.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Conf Matrix
1,0.101100,0.172848,0.962092,0.960066,0.959799,0.962092,"[[3096, 63, 8, 15, 13], [16, 2350, 0, 0, 0], [22, 0, 143, 1, 0], [52, 4, 0, 38, 0], [28, 2, 0, 0, 58]]"
2,0.104500,0.151159,0.960230,0.959102,0.959064,0.960230,"[[3063, 94, 7, 24, 7], [6, 2360, 0, 0, 0], [20, 2, 141, 2, 1], [44, 1, 0, 49, 0], [23, 4, 0, 0, 61]]"
3,0.178800,0.143816,0.966153,0.966017,0.965983,0.966153,"[[3085, 52, 11, 31, 16], [18, 2347, 0, 0, 1], [18, 0, 146, 1, 1], [33, 1, 0, 60, 0], [16, 1, 0, 0, 71]]"


Trainer is attempting to log a value of "[[3096, 63, 8, 15, 13], [16, 2350, 0, 0, 0], [22, 0, 143, 1, 0], [52, 4, 0, 38, 0], [28, 2, 0, 0, 58]]" of type <class 'list'> for key "eval/conf_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[3063, 94, 7, 24, 7], [6, 2360, 0, 0, 0], [20, 2, 141, 2, 1], [44, 1, 0, 49, 0], [23, 4, 0, 0, 61]]" of type <class 'list'> for key "eval/conf_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[[3085, 52, 11, 31, 16], [18, 2347, 0, 0, 1], [18, 0, 146, 1, 1], [33, 1, 0, 60, 0], [16, 1, 0, 0, 71]]" of type <class 'list'> for key "eval/conf_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


TrainOutput(global_step=3324, training_loss=0.17317612418987835, metrics={'train_runtime': 1319.914, 'train_samples_per_second': 40.289, 'train_steps_per_second': 2.518, 'total_flos': 1.3992096579950592e+16, 'train_loss': 0.17317612418987835, 'epoch': 3.0})

In [81]:
#testing mbert swahili model on testing dataset
mbert_test_swahili = mbert_trainer_swahili.evaluate(eval_dataset=swahili_ready_testing)

Trainer is attempting to log a value of "[[2911, 72, 29, 99, 59], [93, 277, 1, 0, 8], [16, 1, 550, 5, 20], [73, 0, 4, 118, 0], [10, 4, 25, 0, 1534]]" of type <class 'list'> for key "eval/conf_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


In [ ]:
#mbert_trainer_swahili.evaluate(eval_dataset=swahili_ready_testing)

In [77]:
mbert_test_english = mbert_trainer_english.evaluate(eval_dataset=english_ready_testing)

Trainer is attempting to log a value of "[[3071, 62, 7, 35, 16], [24, 2284, 0, 2, 3], [29, 2, 161, 1, 3], [36, 2, 2, 66, 2], [20, 1, 0, 0, 80]]" of type <class 'list'> for key "eval/conf_matrix" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


In [78]:
#accessing the logging  history for visualization
training_logs_english = mbert_trainer_english.state.log_history

In [88]:
print(mbert_test_english)

{'eval_loss': 0.20582488179206848, 'eval_accuracy': 0.9581993569131833, 'eval_f1': 0.9578955995479915, 'eval_precision': 0.9579556943722635, 'eval_recall': 0.9581993569131833, 'eval_conf_matrix': [[3071, 62, 7, 35, 16], [24, 2284, 0, 2, 3], [29, 2, 161, 1, 3], [36, 2, 2, 66, 2], [20, 1, 0, 0, 80]], 'eval_runtime': 38.0978, 'eval_samples_per_second': 155.101, 'eval_steps_per_second': 9.712, 'epoch': 3.0}


In [89]:
print(mbert_test_swahili)

{'eval_loss': 0.4714862108230591, 'eval_accuracy': 0.9121678795058386, 'eval_f1': 0.9125165489608603, 'eval_precision': 0.9134012996831977, 'eval_recall': 0.9121678795058386, 'eval_conf_matrix': [[2911, 72, 29, 99, 59], [93, 277, 1, 0, 8], [16, 1, 550, 5, 20], [73, 0, 4, 118, 0], [10, 4, 25, 0, 1534]], 'eval_runtime': 38.0312, 'eval_samples_per_second': 155.372, 'eval_steps_per_second': 9.729, 'epoch': 3.0}
